In [1]:
from setup import *
import numpy as np
from scipy.optimize import curve_fit
import pandas as pd

E0, S0,  all_components = load_models(Egal_reuptake=False)
alpha_table = pd.read_csv('./Data/alpha_table_m2.csv', index_col=0)

initialize E0, S0


# Alter Sij and check flux response in pfba
Analyzing limiting/ reduced cost?

In [2]:
import importlib
import setup
importlib.reload(setup)
from cobra import flux_analysis
from setup import get_BM_df
with E0, S0:
    current_gene = 'dapF'
    alter_Sij(E0, alpha_table.loc[current_gene, 'E0'], current_gene)
    print(E0.optimize())
    
    solution = flux_analysis.pfba(E0)
    print(solution.objective_value)

Variable exists, keep E0, S0
<Solution 0.093 at 0x178820b2390>
398.5892776598805


## Do calculation of drug combination effect

In [4]:
def loglinear(x, a, b, x_start, scale):
    return a*np.exp(b*(x-x_start)/scale)

def loglinear(x, a, b):
    return a*np.exp(b*(x))


# def loglinear(x, b):
#     return 1e-8*np.exp(b*(x))
def truncate_max_biomass(col_str, Biomass_df):
    fit_col = Biomass_df[col_str].dropna()
    cycles_to_fit = fit_col<fit_col.iloc[-1]*.9
    if any(cycles_to_fit.values) == False:
        return None
    
    truncate_col = fit_col.loc[cycles_to_fit]
    # min_tr, max_tr = fit_col[::-1].idxmin(), fit_col.idxmax()
    # truncate_col = fit_col[min_tr :max_tr] # wrong, should consider all for fitting, keep the lag
    # truncate_col = fit_col[:max_tr]
    return truncate_col 

def get_growth_info(col_str, Biomass_df, fit=False, plot_fitted=False, plot_og=False):
    def get_fitted_curve(x, *popt):
        y_fit = loglinear(x, *popt)
        return y_fit
    
    def plot_curves(plot_fitted, plot_og=True, **kwargs):
        if plot_og:
            plt.plot(x, y, label='Original Curve', **kwargs)
        if plot_fitted:        
            y_fit = get_fitted_curve(x,*popt)
            plt.plot(x, y_fit, label='Fitted Curve', **kwargs)
        
    fit_col = truncate_max_biomass(col_str, Biomass_df)
    # Biomass_df[col_str].dropna()
    
    if fit_col is None:
        return [0,0], 0, 0, 0 # popt
    x = fit_col.index
    y= fit_col
    # popt, pcov = curve_fit(loglinear, x, y, p0=[1e-8, .05,0, 2]) # [B,A] -- log(y) = A+Bx
    popt, pcov = curve_fit(loglinear, x, y, p0=[1e-8, .05]) # [B,A] -- log(y) = A+Bx
    popt0, pcov0 = np.polyfit(x, np.log(y), 1) # [B,A] -- log(y) = A+Bx
    
    # popt, pcov = curve_fit(loglinear, x, y, p0=[ .1]) # [B,A] -- log(y) = A+Bx

    plot_curves(plot_fitted, plot_og)

    return popt, pcov, popt0, pcov0
    # return popt, pcov


def plot_filtered_Biomass(g_str, culture, Biomass_df):
    Biomass_df.filter(regex=f'_{g_str}_{culture}').plot()


In [124]:
from collections import defaultdict

# TODO: write into dataclaass
def initialize_growth_rate_dict(method):
    return {method: defaultdict(lambda: defaultdict(dict))}

def get_Biomass_df_from_file(Biomass_df=None, method_suffix=None):
    if method_suffix:
        SG_biomass = pd.read_csv(f'./Data/BM_SG_{method_suffix}.csv', index_col=0)
        DG_biomass = pd.read_csv(f'./Data/BM_DG_{method_suffix}.csv', index_col=0)
        return pd.concat([SG_biomass, DG_biomass], axis=1)
    return Biomass_df
        
def fill_growth_info(Biomass_df=None, method_suffix=None, growth_info=defaultdict(lambda: defaultdict(dict)), n_genes=None):
    def Bliss_Mult(x, y):
        return x * y

    def fill_growth_rate_dict(growth_info, col):
        Species, gcomb, culture = col.split('_')
        sp_cul = '_'.join([Species, culture])


        if sp_cul in growth_info[gcomb]:
            return growth_info[gcomb][sp_cul]

        inner_dict = growth_info[gcomb][sp_cul] # modify this dict below
        inner_dict['popt'], inner_dict['pcov'], inner_dict['popt_np'], inner_dict['pcov_np'] = get_growth_info(col, Biomass_df=Biomass_df)
        # inner_dict['growth_rate'] = inner_dict['popt'][1]
        # inner_dict['growth_rate_err'] = inner_dict['pcov'][1,1]
        # inner_dict['pred_initial_pop'] = inner_dict['popt'][0]

        inner_dict['growth_rate'] = inner_dict['popt'][0]
        inner_dict['growth_rate_np'] = inner_dict['popt_np']
        
        if len(gcomb.split('.')) == 1:
            inner_dict['XG'] = 'SG'
        elif len(gcomb.split('.')) == 2: 
            g1, g2 = gcomb.split('.')
            inner_dict['XG'] = 'DG'

            for current_gene, nth in zip([g1, g2], ['first_gene', 'second_gene']):
                inner_dict[nth] = current_gene
                if sp_cul in growth_info[gcomb]:
                    inner_dict[sp_cul+f'{nth}_gene'] = growth_info[current_gene][sp_cul]['growth_rate']
                # else: 
                #     pass
                    # fill_growth_rate_dict('_'.join([Species, current_gene, culture]))
                
            # inner_dict['Predicted_drug_combination_effect'+sp_cul] = Bliss_Mult(
            #     inner_dict[sp_cul+'first_gene'], inner_dict[sp_cul+'second_gene'])
            # inner_dict['po_diff'+sp_cul] = inner_dict['Predicted_drug_combination_effect'+sp_cul] - inner_dict['growth_rate']
            # inner_dict['po_diff'+sp_cul+'normalized'] = inner_dict['po_diff'+sp_cul]/inner_dict['growth_rate']
        return growth_info
    
    Biomass_df = get_Biomass_df_from_file(Biomass_df=Biomass_df, method_suffix=method_suffix)

    if n_genes is None:
        n_genes = len(Biomass_df.columns)
    for col in Biomass_df.columns[:n_genes]: 
        print(col) 
        fill_growth_rate_dict(growth_info, col)
    return growth_info    
    
def classify_XG(s):
    if len(s.split('.')) == 1:
        return 'SG'
    elif len(s.split('.')) == 2:
        return 'DG'
    
def retrive_growth_rate_value(growth_info):  
    def reorder_rows(df):
        df['XG'] = [classify_XG(s) for s in df.index]        
        df['XG'] = pd.Categorical(df['XG'], categories=['DG', 'SG'])
        return df.sort_values(['XG', 'E0_coculture'], ascending=False)
    
    def get_growth_df(key):
        growth_df = {current_gene: 
                {sp_cul:
                d2[key] for sp_cul, d2 in d1.items()}
                    for current_gene, d1 in growth_info.items()} 
        return growth_df
    
    def get_growth_df_from_dict(key):
        df = pd.DataFrame.from_dict(get_growth_df(key), orient='index')
        df = reorder_rows(df)
        df.index.name = 'Gene_inhibition'
        
        return df
    
    result = {key: get_growth_df_from_dict(key) for key in ['growth_rate', 'growth_rate_np']}

    return result
# ! large error drop x_start
 
# growth_info = defaultdict(dict)
# gr_only = {}
for method_suffix in ['m1', 'm2', 'm3']:
# for method_suffix in [ 'm3']:
    # growth_info.update(initialize_growth_rate_dict(method_suffix))
    # Biomass_df = get_Biomass_df_from_file(method_suffix=method_suffix)
    # growth_info['Biomass'][method_suffix] = Biomass_df
    # growth_info[method_suffix] = fill_growth_info(method_suffix=method_suffix, growth_info=growth_info[method_suffix], n_genes=None)

    gr_only[method_suffix] = retrive_growth_rate_value(growth_info[method_suffix])

    for key in ['growth_rate', 'growth_rate_np']:
        filename_prefix = key.replace('growth_rate', 'gr_SG')
        print(filename_prefix)
        gr_only[method_suffix][key].to_csv(f'./Data/{filename_prefix}_{method_suffix}.csv')

        # with alphatable info
        alpha_table = pd.read_csv(f'./Data/_alpha_table_{method_suffix}.csv', index_col=0)
        pd.concat([gr_only[method_suffix][key], alpha_table], axis=1).to_csv(f'./Data/_{filename_prefix}_{method_suffix}.csv')


gr_SG
gr_SG_np
gr_SG
gr_SG_np
gr_SG
gr_SG_np


In [116]:
def generate_p_o_df(method_n, normalize=False):
    def reorder_columns(df):
        column_order = ['E0_coculture', 'S0_coculture',
       'Predicted_additive_effect_E0_coculture',
       'Predicted_additive_effect_S0_coculture', 'E0_monoculture',
       'S0_monoculture', 'Predicted_additive_effect_E0_monoculture',
       'Predicted_additive_effect_S0_monoculture', 'First_gene',
       'E0_monoculture_First_gene', 'S0_monoculture_First_gene', 'Second_gene',
       'E0_coculture_Second_gene', 'S0_coculture_Second_gene',
       'E0_monoculture_Second_gene', 'S0_monoculture_Second_gene',
       'po_diff_E0_coculture', 'po_diff_S0_coculture',
       'po_diff_E0_monoculture', 'po_diff_S0_monoculture']
        
        return df.reindex(columns=[col for col in column_order if col in df.columns])

    def Bliss_Mult(x, y):
        return x * y
    
    def add_columns(gene_row, SG_df, normalize=False):
        gene_row = gene_row.copy()
        
        cols = gene_row.drop('XG').index
        # add new columns First gene, second gene, predictive additive effect, p-o difference
        if gene_row.name == 'Normal':
            return gene_row
        temp_splitted_gene = gene_row.name.split('.')

        for sp_cult in cols:
            observed_growth_rate = gene_row[sp_cult]
            for current_gene, nth in zip(temp_splitted_gene, ['First_gene', 'Second_gene']):
                gene_row[nth] = current_gene
                # print(current_gene)
                gene_row[sp_cult+f'_{nth}'] = SG_df.loc[current_gene, sp_cult]
                
            if normalize:
                gene_row['Predicted_additive_effect_'+sp_cult] =  Bliss_Mult(*gene_row.filter(regex=f'{sp_cult}_'))
                gene_row['po_diff_'+sp_cult] = gene_row['Predicted_additive_effect_'+sp_cult] - observed_growth_rate
        return gene_row
    
    def save_files(df, normalize):
        SG_df = df.query('XG == "SG"')
        DG_df = df.query('index=="Normal"|XG == "DG"')

        df = DG_df.apply(lambda x: add_columns(x, SG_df=SG_df.copy(), normalize=normalize), axis=1)
        file_suffix = '_normalized' if normalize else ''
        df.index.name = 'Gene_inhibition'

        df.to_csv(f'./Data/gr_DG_{method_n}{file_suffix}.csv')
        return df
    
    type_growth_rate = 'growth_rate_np'
    df = gr_only[method_n][type_growth_rate].copy()

    Normal_growth_rate = df.loc['Normal','E0_coculture':'S0_monoculture']
    df_norm = df.loc[:,'E0_coculture':'S0_monoculture'].div(Normal_growth_rate, axis=1)
    df_norm['XG'] = df['XG']
    
    df = save_files(df, normalize=False)
    df_norm = save_files(df_norm, normalize=True)
    return df_norm if normalize else df

# p3 = generate_p_o_df('m3', normalize=True)
for method_n in ['m1', 'm2', 'm3']:
    gr_only[method_n]['normalized'] = generate_p_o_df(method_n, normalize=True) 

# plots to checking goodness of fit

In [ ]:
# Try dadX only, E should larger than or eq to S
dadX_df = Biomass_df.filter(regex='_dadX_')

growth_rate_d = initialize_growth_rate_dict()
growth_rate_d = get_growth_rate('E0_dadX_coculture', dadX_df, plot_fitted=True, plot_og=True)